#Problem 2: Dimensionality reduction
Consider a dataset where M neurons are recorded during repeated trials of a reach task. The resulting data is a number of matrices (one per trial) that is N ×T (number of neurons by time - in 1 ms bins) where Xit represents the number of spikes for neuron i during the time-bin t. This problem will consider the dimensionality reduction approach of finding the underlying patterns of neural activity common across trials. Specifically we will first look at standard PCA before and after processing (to highlight the benefit of priors over spike-rates). We will then look at applying an external package that finds smooth spike-rates underlying the neural activity automatically (Gaussian Process Factor Analysis).

##Part C
Look in the GPFA folder. Identify how to run GPFA by looking at the demo files
included. Run GPFA on the data with a bin-size of 1ms (instead of the default 20ms). GPFA returs per-trial representations of neural activity. Use the function provided in the gpfa code directory plot3D to plot the time-courses for all of the trials. What do you notice?

In [63]:
!pip install elephant

from elephant.gpfa import GPFA
from google.colab import drive
import numpy as np
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
import matplotlib.pyplot as plt
from quantities import s
from neo.core import SpikeTrain

print("Unfortunately, I could not find the exact Gaussian Process Factor Analysis folder for the assignment.")
print("Thus, I attempted to use another external program named elephant, but was not able to properly get it to work in time.")
print("The sources can be found here: https://elephant.readthedocs.io/en/latest/tutorials/gpfa.html")

# Find the file in google drive
drive.mount('/content/drive')
file = "/content/drive/MyDrive/Neural_Signals_and_Computation_HW2/sample_dat.mat"

import scipy.io as sio
mat_contents = sio.loadmat(file)
data_trials = mat_contents['dat'][0]

## Make Perstimulus Time Histograms
raw_psth = []
bins = np.arange(0, 0.4 + 0.001, 0.001)
for i in range(len(data_trials[0][1])):
  total = []
  for trials in data_trials:
    # This is number of spikes for Neuron i at time bin n (1 msec bins)
    n1 = []
    for j in range(len(trials[1][i])):
      if trials[1][0][j] * 0.001 != 0:
        for k in range(trials[1][i][j]):
          n1.append(0.001 * j)
      elif j == 0 and trials[1][i][j] > 0:
        for k in range(trials[1][i][j]):
          n1.append(0.001 * j)
    if total is not None:
      total += n1
    else:
      total = n1

  y = np.histogram(total, bins=bins)[0] / len(data_trials)
  spike_train = SpikeTrain(list(set(y))*s, t_stop=0.4)
  raw_psth.append(spike_train)
raw_psth = [raw_psth]
# Conduct Gaussian Process
num_trials = 1
timestep = 1 * pq.ms

# specify fitting parameters
bin_size = 1 * pq.ms
latent_dimensionality = 3

gpfa_2dim = GPFA(bin_size=bin_size, x_dim=latent_dimensionality)
gpfa_2dim.fit(raw_psth)
trajectories = gpfa_2dim.transform(raw_psth)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ValueError: Observation covariance matrix is rank deficient.
Possible causes: repeated units, not enough observations.